<a href="https://colab.research.google.com/github/kedrick07/NLP-Semester-5/blob/main/Lab_5_224232_Kedrick.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 5: N-gram Language Processing**
**Course:** SKM3206 – Natural Language Processing  
**Lecturer:** Assoc. Prof. Dr. Azreen Azman / Dr. Nurul Amelina Nasharuddin  
**Due date:** 23 November 2025 (Sunday)

> ⚠️ **Instructions:**  
> 1. Go to *File → Save a copy in Drive* before editing.  
> 2. Rename your notebook as `Lab_LabNo_StudentID_Name.ipynb`.
> 3. Read the examples carefully before attempting the exercises.
> 4. Complete all the tasks in the code cells provided.  
> 5. Submit your Colab link (e.g. http://colab.research.google.com/drive/....) as a submission in the PutraBlast.  





###**💻 Activity**

Review and implement the Python scripts in this [link](https://medium.com/swlh/language-modelling-with-nltk-20eac7e70853) to build an n-gram language model and test it on the next word prediction. Read the 'add-1 smoothing' technique as well. Implement all codes in the code cell below.

---

# 🧮 **Lab Assignment**

### **Instructions:**

1. Modify the codes above to use WSJ corpus as the training  corpus for the n-gram language model.  The WSJ corpus can be can be downloaded from here: https://drive.google.com/file/d/1JMdeLEMC6Zi1XONuDUIYBPJi2M53mcUp/view?usp=drive_link

2. By using *at least 5 incomplete phrases*, evaluate the model for next word prediction.


---
## 📤 **Submission Reminder:**  

Submit the following via PutraBlast:

- The link to your Colab notebook (as *Viewer link*). Ensure the file name is `LabNo_StudentID_Name.ipynb`.
- Output of next word predictions for the 5 strings using the probability models of bigrams, trigrams, and fourgrams, in a text file.
